On importe d'abord le tableau avec toutes les communes J de France et les communes I à moins de 20 minutes ainsi que la durée de trajet. On importe le tableau exporter depuis CalculETP et même chose avec le tableau de CalculPop.

In [ ]:
import pandas as pd

# Charger les trois tableaux Excel dans des DataFrames pandas
df1 = pd.read_excel('CommunesMetrics.xlsx')
df2 = pd.read_excel('chemin_vers_tableau2.xlsx')
df3 = pd.read_excel('chemin_vers_tableau3.xlsx')

On fusionne ces tableaux

In [ ]:
import pandas as pd

# Charger les trois tableaux Excel dans des DataFrames pandas
df1 = pd.read_excel('CommunesMetrics.xlsx')
df2 = pd.read_excel('etp_par_commune.csv')
df3 = pd.read_excel('PopulationSortie.csv')

# Ajouter la colonne "Population" de df1 à df3 avec la condition "Commune" = "Commune I"
df3 = pd.merge(df3, df1[['Commune', 'Population']], left_on='Commune I', right_on='Commune', how='left')

# Renommer la colonne ajoutée
df3 = df3.rename(columns={'Population': 'Population_I'})

# Ajouter la colonne "ETP" de df2 à df3 avec la condition "Commune" = "Commune J"
df3 = pd.merge(df3, df2[['Commune', 'ETP']], left_on='Commune J', right_on='Commune', how='left')

# Renommer la colonne ajoutée
df3 = df3.rename(columns={'ETP': 'ETP_J'})

On défini à présent le coefficient de décroissement en fonction du temps de trajet.

In [ ]:
#Calcul du décroissement de distance
def calcul_decroissement(row):
    trajet = row['Trajet']
    if trajet < 10:
        return row['Population_J'] * 1
    elif 10 <= trajet < 15:
        return row['Population_J'] * (2 / 3)
    elif 15 <= trajet <= 20:
        return row['Population_J'] * (2 / 3)
    else:
        return None

# Appliquer la fonction pour créer la colonne "Décroissement"
df3['Décroissement'] = df3.apply(calcul_decroissement, axis=1)

On calcul à présent le ration de chaque commune J

In [ ]:
#Calcul du ratio
  # Groupez les données par Commune J
grouped_df = df.groupby('Commune J')

# Initialisez une liste pour stocker les résultats
resultats = []

# Parcourez chaque groupe
for commune, group in grouped_df:
    # Calculez l'ETP de la commune actuelle
    etp_commune = group['ETP_J'].iloc[0]
    
    # Calculez la somme des Décroissements pour cette commune
    decroissement_sum = group['Décroissement'].sum()
    
    # Effectuez le calcul final
    ratio = etp_commune / decroissement_sum
    
    # Ajoutez le résultat à la liste
    resultats.append({'Commune J': commune, 'Ratio J': ratio})

# Créez un DataFrame avec les résultats
resultats_df = pd.DataFrame(resultats)

# Fusionnez resultats_df avec le DataFrame d'origine df
df = pd.merge(df, resultats_df, on='Commune J', how='left')

# Affichez le DataFrame résultant avec la nouvelle colonne "Ratio J"
print(df)


On calcul à présent l'APL pour chaque commune J

In [2]:
#Calcul APL pour chaque commune I 

# Groupez les données par Commune I et calculez la somme des produits des ratios par Décroissement
result_sum = df.groupby('Commune I').apply(lambda x: (x['Ratio J'] * x['Décroissement']).sum()).reset_index()

# Renommez les colonnes
result_sum.columns = ['Commune I', 'APL']

# Affichez le résultat
print(result_sum)




On exporte ensuite en CSV pour être utiliser par l'appli web

In [ ]:
# Exporter le DataFrame result_sum vers un fichier CSV
result_sum.to_csv('result_sum.csv', index=False)